Django provides a powerful and flexible framework for creating and validating forms. 
Understanding forms and validation is crucial for building dynamic web applications that handle user input securely and effectively.

### What Are Forms in Django?

Django forms are Python classes that represent an HTML form. They allow you to define input fields, validate user input, and process data efficiently. Django provides:

Built-in form handling for user input.
Validation mechanisms to ensure secure and accurate data handling.

## Creating and Using Forms

### 1. Creating a Basic Form

Define a form in forms.py:

In [ ]:
# forms.py
from django import forms

class ContactForm(forms.Form):
    name = forms.CharField(max_length=100, label="Your Name")
    email = forms.EmailField(label="Your Email")
    message = forms.CharField(widget=forms.Textarea, label="Message")


Fields: Each field (e.g., CharField, EmailField) corresponds to an HTML input element.

Attributes:

max_length: Maximum number of characters for the field.

widget: Specifies the HTML widget (e.g., textarea).

label: Customizes the field's label in the form.

### 2. Rendering the Form in a View

Use the form in a Django view:

In [ ]:
# views.py
from django.shortcuts import render
from .forms import ContactForm

def contact_view(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            # Process form data
            print(form.cleaned_data)
    else:
        form = ContactForm()

    return render(request, 'contact.html', {'form': form})


request.POST: Contains data submitted via POST.

form.is_valid(): Checks if the form passes validation.

form.cleaned_data: Dictionary of valid form data.

### 3. Rendering the Form in a Template

Render the form using the template language:

In [ ]:
<!-- templates/contact.html -->
<!DOCTYPE html>
<html>
<head>
    <title>Contact Us</title>
</head>
<body>
    <h1>Contact Us</h1>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Send</button>
    </form>
</body>
</html>


{% csrf_token %}: Protects against Cross-Site Request Forgery.

form.as_p: Renders the form fields as <p> elements

## Customizing Forms

### 1. Adding CSS Classes
You can add custom CSS classes or attributes to fields:

In [ ]:
# forms.py
class ContactForm(forms.Form):
    name = forms.CharField(
        max_length=100,
        widget=forms.TextInput(attrs={'class': 'form-control', 'placeholder': 'Enter your name'})
    )
    email = forms.EmailField(
        widget=forms.EmailInput(attrs={'class': 'form-control', 'placeholder': 'Enter your email'})
    )
    message = forms.CharField(
        widget=forms.Textarea(attrs={'class': 'form-control', 'rows': 5, 'placeholder': 'Your message'})
    )


### 2. Customizing Labels and Help Text

Add labels or help text for better user guidance:

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(label="Full Name", help_text="Enter your full name")
    email = forms.EmailField(label="Email Address", help_text="We'll never share your email.")
    message = forms.CharField(label="Message", widget=forms.Textarea, help_text="Enter your message.")


## Form Validation

Django provides multiple ways to validate form data.

### 1. Built-in Validation

Django fields come with built-in validation:

EmailField ensures a valid email.
CharField respects max_length.

### 2. Custom Validation

a) Field-Level Validation:

Define a clean_fieldname method to validate a specific field.

In [ ]:
class ContactForm(forms.Form):
    email = forms.EmailField()
    
    def clean_email(self):
        email = self.cleaned_data.get('email')
        if not email.endswith('@example.com'):
            raise forms.ValidationError("Email must be from @example.com domain.")
        return email


### b) Form-Level Validation:

Override the clean method to validate multiple fields together.

In [ ]:
class ContactForm(forms.Form):
    name = forms.CharField(max_length=100)
    email = forms.EmailField()
    
    def clean(self):
        cleaned_data = super().clean()
        name = cleaned_data.get('name')
        email = cleaned_data.get('email')
        
        if "admin" in name.lower() and email.endswith("@example.com"):
            raise forms.ValidationError("Admin emails cannot use @example.com domain.")


### 3. Error Handling in Templates
Display form errors

In [ ]:
<form method="post">
    {% csrf_token %}
    {% for field in form %}
        <div>
            {{ field.label_tag }} {{ field }}
            {% if field.errors %}
                <ul>
                {% for error in field.errors %}
                    <li>{{ error }}</li>
                {% endfor %}
                </ul>
            {% endif %}
        </div>
    {% endfor %}
    <button type="submit">Submit</button>
</form>


## Model Forms

Model forms simplify creating forms tied to a database model.

### 1. Creating a Model Form

Define a model:

In [ ]:
# models.py
from django.db import models

class Contact(models.Model):
    name = models.CharField(max_length=100)
    email = models.EmailField()
    message = models.TextField()


Create a form from the model:

In [ ]:
# forms.py
from django.forms import ModelForm
from .models import Contact

class ContactForm(ModelForm):
    class Meta:
        model = Contact
        fields = ['name', 'email', 'message']


### 2. Using the Model Form

In the view:

In [ ]:
# views.py
from django.shortcuts import render
from .forms import ContactForm

def contact_view(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            form.save()  # Saves the data to the database
    else:
        form = ContactForm()

    return render(request, 'contact.html', {'form': form})


## Dynamic Formsets

Django formsets allow managing multiple forms on a single page.

Example: Dynamic Contact Forms

In [ ]:
from django.forms import formset_factory
from .forms import ContactForm

def contact_formset_view(request):
    ContactFormSet = formset_factory(ContactForm, extra=2)  # 2 empty forms
    formset = ContactFormSet()

    return render(request, 'formset_example.html', {'formset': formset})


# Summary:

## Basic Forms:

Defined using forms.Form.
Handle user input and perform basic validation.
Customization:

Add custom attributes, labels, and help texts.
Apply CSS classes for styling.

## Validation:

Use built-in or custom validation methods.
Handle form-level and field-level validations.

## Model Forms:

Simplify form creation for models.
Automatically map fields to database models.

## Error Handling:

Display errors dynamically in templates for better user feedback.
By mastering forms and validation in Django, you can create robust, user-friendly web applications that securely handle input!